# MoA Prediction

In [1]:
import os
import pickle

import numpy as np
import pandas as pd
import timm
import torch
import torch.nn as nn
from albumentations import CenterCrop, Compose, Normalize
from albumentations.pytorch import ToTensorV2
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (average_precision_score, roc_auc_score)
from sklearn.model_selection import GroupKFold, KFold
from torch.utils.data import DataLoader
from tqdm import tqdm

from toxreprcnn.dataset import ToxReprCNNDataset
from toxreprcnn.model import EffnetB4ModelMO, FrozenEffnetB4ModelMO
from toxreprcnn.utils import fix_seed

root = ".."

In [2]:
moa_df = pd.read_csv(f"{root}/data/TGGATEs/processed/moa.csv").rename(columns={"Unnamed: 0":"COMPOUND_NAME"})
moa_df

,COMPOUND_NAME,Bacterial 70S ribosome inhibitor,Serotonin 2a (5-HT2a) receptor antagonist,DNA inhibitor,Cyclooxygenase inhibitor,"Sulfonylurea receptor 1, Kir6.2 blocker",Histamine H2 receptor antagonist,Peroxisome proliferator-activated receptor alpha agonist
0,erythromycin ethylsuccinate,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,acetaminophen,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,chlorpromazine,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ranitidine,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,azathioprine,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,haloperidol,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,carboplatin,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,phenylbutazone,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,cyclophosphamide,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,naproxen,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [3]:
info = pd.read_csv(f"{root}/data/TGGATEs/processed/info.csv")

moa_df["MoA"] = np.argmax(moa_df[moa_df.columns[1:]].values,axis=1)
moa_info = pd.merge(info, moa_df, on = "COMPOUND_NAME")

moa_info_selected = moa_info[moa_info["SACRI_PERIOD"].isin(["4 day", "8 day", "15 day", "29 day"]) & (moa_info["DOSE"]>0)]
moa_info_selected["EG"] = moa_info_selected["EXP_ID"]*100+moa_info_selected["GROUP_ID"]

moa_info_selected

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,EXP_ID,GROUP_ID,INDIVIDUAL_ID,INDV_ID,COMPOUND_NAME,COMPOUND_ABBR,COMPOUND_NO,SPECIES,TEST_TYPE,SIN_REP_TYPE,...,FILE_LOCATION,Bacterial 70S ribosome inhibitor,Serotonin 2a (5-HT2a) receptor antagonist,DNA inhibitor,Cyclooxygenase inhibitor,"Sulfonylurea receptor 1, Kir6.2 blocker",Histamine H2 receptor antagonist,Peroxisome proliferator-activated receptor alpha agonist,MoA,EG
106,42,9,1,42091,acetaminophen,APAP,1,Rat,in vivo,Repeat,...,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3,4209
107,42,9,2,42092,acetaminophen,APAP,1,Rat,in vivo,Repeat,...,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3,4209
108,42,9,3,42093,acetaminophen,APAP,1,Rat,in vivo,Repeat,...,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3,4209
109,42,9,4,42094,acetaminophen,APAP,1,Rat,in vivo,Repeat,...,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3,4209
110,42,9,5,42095,acetaminophen,APAP,1,Rat,in vivo,Repeat,...,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3,4209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5030,577,16,1,577161,carboplatin,CBP,133,Rat,in vivo,Repeat,...,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2,57716
5031,577,16,2,577162,carboplatin,CBP,133,Rat,in vivo,Repeat,...,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2,57716
5032,577,16,3,577163,carboplatin,CBP,133,Rat,in vivo,Repeat,...,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2,57716
5033,577,16,4,577164,carboplatin,CBP,133,Rat,in vivo,Repeat,...,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2,57716


In [4]:
train = pd.read_csv(f"../../data/TGGATEs/processed/train.csv")
moa_info_test = moa_info_selected[~moa_info_selected["EG"].isin(train["EG"].unique())]
moa_info_test

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,EXP_ID,GROUP_ID,INDIVIDUAL_ID,INDV_ID,COMPOUND_NAME,COMPOUND_ABBR,COMPOUND_NO,SPECIES,TEST_TYPE,SIN_REP_TYPE,...,FILE_LOCATION,Bacterial 70S ribosome inhibitor,Serotonin 2a (5-HT2a) receptor antagonist,DNA inhibitor,Cyclooxygenase inhibitor,"Sulfonylurea receptor 1, Kir6.2 blocker",Histamine H2 receptor antagonist,Peroxisome proliferator-activated receptor alpha agonist,MoA,EG
106,42,9,1,42091,acetaminophen,APAP,1,Rat,in vivo,Repeat,...,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3,4209
107,42,9,2,42092,acetaminophen,APAP,1,Rat,in vivo,Repeat,...,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3,4209
108,42,9,3,42093,acetaminophen,APAP,1,Rat,in vivo,Repeat,...,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3,4209
109,42,9,4,42094,acetaminophen,APAP,1,Rat,in vivo,Repeat,...,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3,4209
110,42,9,5,42095,acetaminophen,APAP,1,Rat,in vivo,Repeat,...,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3,4209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5030,577,16,1,577161,carboplatin,CBP,133,Rat,in vivo,Repeat,...,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2,57716
5031,577,16,2,577162,carboplatin,CBP,133,Rat,in vivo,Repeat,...,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2,57716
5032,577,16,3,577163,carboplatin,CBP,133,Rat,in vivo,Repeat,...,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2,57716
5033,577,16,4,577164,carboplatin,CBP,133,Rat,in vivo,Repeat,...,ftp://ftp.biosciencedbc.jp/archive/open-tggate...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2,57716


In [5]:
from glob import glob
moa_tiles = []
for v in moa_info_test["FILE"].to_numpy():
    if type(v) == type(""):
        moa_tiles += glob("/data/TGGATE/tiles/"+v+"/*.tiff")


In [6]:
len(moa_tiles)

55026

In [7]:
seed = 123
fix_seed(seed)

In [8]:
image_size = 512

vl_transform = Compose([CenterCrop(image_size, image_size), Normalize(), ToTensorV2()])


In [9]:
test_dataset = ToxReprCNNDataset(
    moa_tiles, [0]*len(moa_tiles), transform=vl_transform
)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    drop_last=False,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
)

model_control = timm.create_model("tf_efficientnet_b4_ns", pretrained=True, num_classes=0)
model_control.eval()
model_control.to("cuda")

ft_list = [None]*8

models_multiseed = []
for seed in range(123,128):
    save_dir = f"../../outputs/TGGATEs_model_seed{seed}"
    models = [FrozenEffnetB4ModelMO(i, len(ft_list)) for i in range(8)] + [EffnetB4ModelMO(num_classes=len(ft_list))]
    for i, model in enumerate(models):
        if i <= 7:
            model.load_state_dict(
                torch.load(f"{save_dir}/{i}/effnetb4_freeze{i}_fold0_best_loss.pth")
            )
            model.classifier = nn.Identity()
        else:
            model.model.load_state_dict(
                torch.load(f"{save_dir}/{i}/effnetb4_freeze{i}_fold0_best_loss.pth")
            )
            model.model.classifier = nn.Identity()
        model.to("cuda")
        model.eval()
    models_multiseed.append(models)

In [11]:
features_control = []
features = [[[[] for k in range(9)] for i in range(9)] for seed in range(5)]
with torch.no_grad():
    for im, _ in tqdm(test_loader):
        im = im.to("cuda")
        outputs = model_control(im)
        features_control.append(outputs.to("cpu").numpy())
        for seed in range(5):
            for j in range(9):
                outputs = models_multiseed[seed][j](im)
                for k, f in enumerate(outputs):
                    features[seed][j][k].append(f.to("cpu").numpy())


100%|██████████| 1720/1720 [3:07:46<00:00,  6.55s/it]  


In [12]:
with open(f"{root}/outputs/compound_validation_features.pickle", "wb") as f:
    pickle.dump({"features" : features, "features_control" : features_control, "moa_tiles" : moa_tiles}, f)

In [15]:
with open(f"{root}/outputs/compound_validation_features.pickle", "rb") as f:
    data = pickle.load(f)

features = data["features"]
features_control = data["features_control"]
moa_tiles = data["moa_tiles"]

In [19]:
ft_all_list = moa_info_test.columns[77:77+65]
ft_all_list


Index(['Accumulation, foam cell', 'Adenoma, hepatocellular',
       'Alteration, cytoplasmic', 'Alteration, nuclear',
       'Altered hepatocellular foci', 'Anisonucleosis', 'Atrophy',
       'Atypia, nuclear', 'Bacterium', 'Cellular foci',
       'Cellular infiltration', 'Cellular infiltration, mononuclear cell',
       'Cellular infiltration, neutrophil', 'Change, acidophilic',
       'Change, basophilic', 'Change, eosinophilic', 'Congestion', 'Cyst',
       'DEAD', 'Degeneration', 'Degeneration, acidophilic, eosinophilic',
       'Degeneration, fatty', 'Degeneration, granular',
       'Degeneration, granular, eosinophilic', 'Degeneration, hydropic',
       'Degeneration, vacuolar', 'Deposit, glycogen', 'Deposit, hemosiderin',
       'Deposit, lipid', 'Deposit, pigment', 'Dilatation', 'Disarrangement',
       'Ectopic tissue', 'Edema', 'Fibrosis', 'Giant cell', 'Granuloma',
       'Ground glass appearance', 'Hematopoiesis, extramedullary',
       'Hemorrhage', 'Hyperplasia', 'Hypertr

In [21]:
def labeler(y_train, y_valid):
    y_list = list(set(list(y_train)))
    y_list.sort()
    y_dict = {v:k for k,v in enumerate(y_list)}
    return np.array([y_dict[v] if v in y_dict else 100 for v in y_valid])

ft_list = ['Proliferation, bile duct',
 'Ground glass appearance',
 'Increased mitosis',
 'Inclusion body, intracytoplasmic',
 'Deposit, pigment',
 'Single cell necrosis',
 'Vacuolization, cytoplasmic',
 'Swelling']

def create_fold(moa_info_test_f, seed):
    kf = KFold(n_splits=5, shuffle=True, random_state=seed)
    eg = moa_info_test_f["EG"].unique()
    fold = np.zeros(len(eg))
    for i, (_, idx) in enumerate(kf.split(eg)):
        fold[idx] = i
    fold1 = np.zeros(len(moa_info_test_f))
    for i in range(5):
        fold1[moa_info_test_f["EG"].isin(eg[fold==i])] = i
    return fold1

def compound_validation(features, feature_list, moa_tiles, fold):
    macro_acc = []
    acc = []
    auroc = []
    mAP = []
    if features is not None:
        df = pd.DataFrame(features)
        df["FILE"] = [t.split("/")[-2] for t in moa_tiles]
        df = df.groupby("FILE").mean()
        moa_info_test_f = pd.merge(moa_info_test, df, on = "FILE", how="inner")
    else:
        moa_info_test_f = moa_info_test

    for i in range(5):
        X = moa_info_test_f[feature_list].values
        y = moa_info_test_f["MoA"].values
        X_train = X[fold!=i]
        y_train = y[fold!=i]
        X_valid = X[fold==i]
        y_valid = y[fold==i]
        lr = LogisticRegression(max_iter=10000, n_jobs=8)
        lr.fit(X_train, y_train)
        y_preds = lr.predict_proba(X_valid)
        y_valid = labeler(y_train, y_valid)
        macro_acc.append(balanced_accuracy_score(y_valid, np.argmax(y_preds, axis=1)))
        acc.append([balanced_accuracy_score(y_valid==i, np.argmax(y_preds, axis=1)==i) \
            if (y_valid==i).sum()!=0 else np.nan for i in range(max(moa_info_test_f["MoA"])+1)])
        auroc.append([roc_auc_score(y_valid==i, y_preds[:, i]) \
            if (y_valid==i).sum()!=0 else np.nan for i in range(max(moa_info_test_f["MoA"])+1)])
        mAP.append([(average_precision_score(y_valid==i, y_preds[:, i]) +  \
                    average_precision_score(y_valid!=i, - y_preds[:, i])) / 2 \
            if (y_valid==i).sum()!=0 else np.nan for i in range(max(moa_info_test_f["MoA"])+1)])
    return macro_acc, acc, auroc, mAP


df = pd.DataFrame(np.concatenate(features[0][0][0]))
df["FILE"] = [t.split("/")[-2] for t in moa_tiles]
df = df.groupby("FILE").mean()
_moa_info_test_f = pd.merge(moa_info_test, df, on = "FILE", how="inner")


macro_acc_res = [[[[] for j in range(9)] for i in range(10)] for seed in range(5)]
acc_res = [[[[] for j in range(9)] for i in range(10)] for seed in range(5)]
auroc_res = [[[[] for j in range(9)] for i in range(10)] for seed in range(5)]
mAP_res = [[[[] for j in range(9)] for i in range(10)] for seed in range(5)]
for _ in range(10):
    fold = create_fold(_moa_info_test_f, 123+_)
    for seed in range(5):
        features_temp = [[features[seed][0][i] for i in range(8)] + [features_control]] + features[seed]
        for i in range(10):
            for j in tqdm(range(9)):
                macro_acc, acc, auroc, mAP = compound_validation(np.concatenate(features_temp[i][j]), range(np.concatenate(features_temp[i][j]).shape[1]), moa_tiles, fold)
                macro_acc_res[seed][i][j].append(macro_acc)
                acc_res[seed][i][j].append(acc)
                auroc_res[seed][i][j].append(auroc)           
                mAP_res[seed][i][j].append(mAP) 

  0%|          | 0/9 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
 11%|█         | 1/9 [00:06<00:54,  6.79s/it]/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains cl

In [22]:
ft_macro_acc = []
ft_acc = []
ft_auroc = []
ft_mAP = []
for _ in tqdm(range(10)):
    fold = create_fold(_moa_info_test_f, 123+_)
    macro_acc, acc, auroc, mAP = compound_validation(np.concatenate(features_temp[0][0]), ft_all_list, moa_tiles, fold)
    ft_macro_acc.append(macro_acc)
    ft_acc.append(acc)
    ft_auroc.append(auroc)
    ft_mAP.append(mAP)

 30%|███       | 3/10 [00:00<00:01,  4.13it/s]/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
 50%|█████     | 5/10 [00:01<00:01,  4.08it/s]/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
 80%|████████  | 8/10 [00:01<00:00,  4.08it/s]/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains c

In [23]:
len(macro_acc_res)

5

In [33]:
model_name = ["Control"] + ["Head"] + [f"Block {7-i}" for i in range(7)] + ["Full"]
layer_name = ["Stem"] + [f"Block {i+1}" for i in range(7)] + ["Head"]

for seed in range(5):
    rec = []
    for i in range(10):
        for j in range(9):
            for k in range(10):
                for l in range(5):        
                    rec.append([model_name[i], layer_name[j],k,l, macro_acc_res[seed][i][j][k][l]])
    for i in range(10):
        for j in range(5):
            rec.append(["Pathological Findings", "-",i,j, ft_macro_acc[i][j]])
    macro_raw_df = pd.DataFrame(rec, columns=["model", "layer", "trial", "fold", "Macro Balanced Accuracy"])
    macro_raw_df.to_csv(f"{root}/outputs/results/compound_validation_macro_{123+seed}.csv", index=False)


In [27]:
moa_df.columns[1:8]

Index(['Bacterial 70S ribosome inhibitor',
       'Serotonin 2a (5-HT2a) receptor antagonist', 'DNA inhibitor',
       'Cyclooxygenase inhibitor', 'Sulfonylurea receptor 1, Kir6.2 blocker',
       'Histamine H2 receptor antagonist',
       'Peroxisome proliferator-activated receptor alpha agonist'],
      dtype='object')

In [31]:
moa_list = moa_df.columns[1:8].to_list()

In [43]:
model_name = ["Control"] + ["Head"] + [f"Block {7-i}" for i in range(7)] + ["Full"]
layer_name = ["Stem"] + [f"Block {i+1}" for i in range(7)] + ["Head"]
for seed in range(5):
    rec = []
    for i in range(10):
        for j in range(9):
            for k in range(10):
                for l in range(5):        
                    rec.append([model_name[i], layer_name[j],k,l] + acc_res[seed][i][j][k][l] + \
                               auroc_res[seed][i][j][k][l] + \
                                mAP_res[seed][i][j][k][l])
    for i in range(10):
        for j in range(5):
            rec.append(["Pathological Findings", "-",i,j] + ft_acc[i][j] + ft_auroc[i][j] + ft_mAP[i][j])
    raw_df = pd.DataFrame(rec, columns=["model", "layer", "trial", "fold"] + [m + "_Balanced Accuracy" for m in moa_list] + \
                          [m + "_AUROC" for m in moa_list] + \
                            [m + "_mAP" for m in moa_list])
    raw_df.to_csv(f"{root}/outputs/results/compound_validation_{123+seed}.csv", index=False) 